In [1]:
# import libraries
import pandas as pd
import numpy as np
import ast
import json
import warnings
from random import sample
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

In [2]:
call_data = pd.read_csv('call_inference_agg.csv')
call_data.head(10)

,rexurl,call_ct,buyer_call
0,1-50th-st-1134,1,0
1,1-anchorage-way,1,0
2,1-brook-dr-s,12,5
3,1-buckingham-ct,9,1
4,1-canterbury-ct,2,0
5,1-channel-dr-unit-304,1,0
6,1-charles-davis-dr,1,0
7,1-concord-ave,1,0
8,1-emerald-bay-dr,1,0
9,1-fern-way,26,1


In [3]:
web_data = pd.read_csv('web_events_result.csv')
web_data.head()

,id,event_stamp,action,category,ip,visitor_guid,url,city_name,country_code2,domain,...,listingGuid,longitude,postal_code,propertyGuid,region_code,region_name,rexUrl,sessionGuid,uriStem,userGuid
0,109188722,2020-09-04 20:17:51.407000,pageview,NaN,98.148.1.185,3c41a04d70044e49bbccead33388733f,https://www.rexhomes.com/compare/rex,NaN,US,www.rexhomes.com,...,NaN,-97.822,NaN,NaN,NaN,NaN,NaN,874173460ea74486be409dad6a01f60e,/compare/rex,NaN
1,109188735,2020-09-04 20:17:55.184000,pageview,NaN,98.148.1.185,3c41a04d70044e49bbccead33388733f,https://www.rexhomes.com/compare/rex,NaN,US,www.rexhomes.com,...,NaN,-97.822,NaN,NaN,NaN,NaN,NaN,9340b9948477447b87f7be4f73f0d8d9,/compare/rex,NaN
2,109211987,2020-09-04 23:00:37.745000,pageview,NaN,98.148.1.185,3c41a04d70044e49bbccead33388733f,https://www.rexhomes.com/sell-with-rex,NaN,US,www.rexhomes.com,...,NaN,-97.822,NaN,NaN,NaN,NaN,NaN,aa88596c4de049db8121e7b746844fce,/sell-with-rex,NaN
3,109211992,2020-09-04 23:00:39.676000,pageview,NaN,98.148.1.185,3c41a04d70044e49bbccead33388733f,https://www.rexhomes.com/sell-with-rex,NaN,US,www.rexhomes.com,...,NaN,-97.822,NaN,NaN,NaN,NaN,NaN,72f07640b6c74bcf8603fd8c8ab608a4,/sell-with-rex,NaN
4,108604646,2020-09-01 00:07:05.944000,pageview,NaN,74.125.150.85,bd73ee3ba36f45858d3e967cba74f006,https://www.rexhomes.com/sell-with-rex?utm_sou...,NaN,US,www.rexhomes.com,...,NaN,-97.822,NaN,NaN,NaN,NaN,NaN,acad39fc61b94bfbb2bda419efd207b1,/sell-with-rex,NaN


In [4]:
# list address
address_data = pd.read_csv("listings_addr.csv")
address_data.head(2)

,Unnamed: 0,id_x,guid_x,address_line,city,state,postal_code,created_x,updated_x,id_y,...,sales,school_district_id,school_district_name,status,stories,timezone_id,total_baths,total_baths_calculated,year_built,addr_lower_x
0,0,360,75bb16cdde1e40d3bacfcde645ffa1b2,1245 Cordova St Apt 301,Pasadena,CA,91106,2016-05-21 15:30:03,2016-05-21 15:30:03,459,...,LA,NaN,NaN,INACTIVE,0.0,America/Los_Angeles,2.0,2.0,1987.0,1245 cordova st apt 301
1,1,361,27972de492f4441fbc62a55908b9ba79,216 S Madison Ave Apt 205,Pasadena,CA,91101,2016-05-21 16:44:31,2016-05-21 16:44:31,460,...,LA,NaN,NaN,PREVIEW,0.0,America/Los_Angeles,2.0,2.0,2001.0,216 s madison ave apt 205


In [5]:
# address with call data
print("Origin data: ", call_data.shape, address_data.shape)
merge_df = address_data.merge(call_data, left_on='rex_url', right_on="rexurl", how='left')
merge_df.head(5)

Origin data:  (6969, 3) (128998, 37)


,Unnamed: 0,id_x,guid_x,address_line,city,state,postal_code,created_x,updated_x,id_y,...,status,stories,timezone_id,total_baths,total_baths_calculated,year_built,addr_lower_x,rexurl,call_ct,buyer_call
0,0,360,75bb16cdde1e40d3bacfcde645ffa1b2,1245 Cordova St Apt 301,Pasadena,CA,91106,2016-05-21 15:30:03,2016-05-21 15:30:03,459,...,INACTIVE,0.0,America/Los_Angeles,2.0,2.0,1987.0,1245 cordova st apt 301,1245-cordova-st-unit-301,12.0,3.0
1,1,361,27972de492f4441fbc62a55908b9ba79,216 S Madison Ave Apt 205,Pasadena,CA,91101,2016-05-21 16:44:31,2016-05-21 16:44:31,460,...,PREVIEW,0.0,America/Los_Angeles,2.0,2.0,2001.0,216 s madison ave apt 205,NaN,NaN,NaN
2,2,362,842b64bc81f4421888274b55cb15228c,2355 Grand Ave,Ventura,CA,93003,2016-05-22 20:33:15,2016-05-22 20:33:15,461,...,PREVIEW,1.0,America/Los_Angeles,1.0,1.0,1949.0,2355 grand ave,NaN,NaN,NaN
3,3,363,5dd958430d54481c98863d32337e8c6c,6034 Capistrano Ave,Woodland Hills,CA,91367,2016-05-22 23:50:42,2016-05-22 23:50:42,462,...,PREVIEW,1.0,America/Los_Angeles,3.0,3.0,1965.0,6034 capistrano ave,NaN,NaN,NaN
4,4,364,a74433fdd2b3436fb12a504e5e3e3204,15206 Hartsook St,Sherman Oaks,CA,91403,2016-05-23 02:23:13,2016-05-23 02:23:13,463,...,PREVIEW,1.0,America/Los_Angeles,5.0,5.0,1945.0,15206 hartsook st,NaN,NaN,NaN


In [6]:
# merge web event to address+call data
web_df_wi_rexurl = web_data.loc[web_data.rexUrl.notnull()]
print("Origin data: ", merge_df.shape, web_df_wi_rexurl.shape)
merge_df2 = web_df_wi_rexurl.merge(merge_df, left_on='rexUrl', right_on='rex_url', how='left')
print("After:", merge_df2.shape)
merge_df2.head(10)

Origin data:  (128998, 40) (845579, 23)
After: (845579, 63)


,id,event_stamp,action,category,ip,visitor_guid,url,city_name,country_code2,domain,...,status,stories,timezone_id,total_baths,total_baths_calculated,year_built,addr_lower_x,rexurl,call_ct,buyer_call
0,108606511,2020-09-01 00:15:50.547000,ViewContent,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/1511-fieldvie...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,2.5,3.0,2006.0,1511 fieldview dr,1511-fieldview-dr-jacksonville,3.0,0.0
1,108606512,2020-09-01 00:15:50.755000,view_item,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/1511-fieldvie...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,2.5,3.0,2006.0,1511 fieldview dr,1511-fieldview-dr-jacksonville,3.0,0.0
2,108606513,2020-09-01 00:15:50.963000,pageview,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/1511-fieldvie...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,2.5,3.0,2006.0,1511 fieldview dr,1511-fieldview-dr-jacksonville,3.0,0.0
3,108606519,2020-09-01 00:15:53.437000,ViewContent,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/9613-glendowe...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,3.5,4.0,1989.0,9613 glendower ct,NaN,NaN,NaN
4,108606520,2020-09-01 00:15:53.645000,view_item,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/9613-glendowe...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,3.5,4.0,1989.0,9613 glendower ct,NaN,NaN,NaN
5,108606521,2020-09-01 00:15:53.853000,pageview,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/9613-glendowe...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,3.5,4.0,1989.0,9613 glendower ct,NaN,NaN,NaN
6,108607076,2020-09-01 00:19:18.586000,ViewContent,NaN,74.125.150.86,275b4f8b46f443bb9d619a4038cf776e,https://www.rexhomes.com/listing/594-perry-st?...,NaN,US,www.rexhomes.com,...,FOR_SALE,1.0,America/New_York,2.0,2.0,2004.0,594 perry st,NaN,NaN,NaN
7,108607077,2020-09-01 00:19:18.813000,view_item,NaN,74.125.150.86,275b4f8b46f443bb9d619a4038cf776e,https://www.rexhomes.com/listing/594-perry-st?...,NaN,US,www.rexhomes.com,...,FOR_SALE,1.0,America/New_York,2.0,2.0,2004.0,594 perry st,NaN,NaN,NaN
8,108607078,2020-09-01 00:19:19.021000,pageview,NaN,74.125.150.86,275b4f8b46f443bb9d619a4038cf776e,https://www.rexhomes.com/listing/594-perry-st?...,NaN,US,www.rexhomes.com,...,FOR_SALE,1.0,America/New_York,2.0,2.0,2004.0,594 perry st,NaN,NaN,NaN
9,108607830,2020-09-01 00:24:23.658000,ViewContent,NaN,74.125.150.82,32305be402e74dfab9d475fb2d76235c,https://www.rexhomes.com/listing/9613-glendowe...,NaN,US,www.rexhomes.com,...,SOLD,2.0,America/New_York,3.5,4.0,1989.0,9613 glendower ct,NaN,NaN,NaN


In [7]:
# clean data
# remaining features (ip, country_code2, eventAttributes, latitude_y, longitude_y, city, state, 
# full_baths, bedrooms, land_square_footage, living_square_feet, marketing_budget - categorical, 
# property_type - categorical, school_district_name - categorical, status, stories, total_baths, 
# year_built, call_ct, buyer_call)
# now we consider houses with status of both sold and for sale

selected_features = ['ip','country_code2', 'eventAttributes', 'latitude_y', 'longitude_y', 'city', 'state',\
                    'full_baths', 'bedrooms', 'land_square_footage', 'living_square_feet', 'marketing_budget',\
                    'property_type', 'school_district_name', 'status', 'stories', 'total_baths',\
                    'year_built', 'call_ct', 'buyer_call', 'rexUrl']
cat_features = ['country_code2', 'city', 'state', 'marketing_budget', 'property_type',\
                'school_district_name', 'status']


In [8]:
def transformer(df, feature_name):
    '''
    Params:
        df -> pandas dataframe: the original dataframe containing the categorical feature
        feature_name -> str: the categorical feature to be transformed
    Returns:
        transformed_feature -> pandas dataframe: one hot encoded feature
    '''
    transformed_feature = pd.get_dummies(df[feature_name], prefix=feature_name)
    return transformed_feature

In [9]:
def join(df, transformed_df):
    '''
    Params:
        df -> pandas dataframe: the original dataframe containing the categorical feature
        transformed_df -> pandas dataframe: one hot encoded dataframe
    Returns:
        joined_df -> pandas dataframe: joined dataframe of original dataframe and 
                                       the transformed dataframe on the categorical feature
    '''
    joined_df = df.join(transformed_df)
    return joined_df

In [10]:
def drop(df, cat_feature):
    '''
    Params:
        df -> pandas dataframe: the joined dataframe containing the categorical feature
        cat_feature -> str: the categorical feature to be dropped
    Returns:
        dropped_df -> pandas dataframe: the dataframe with the categorical feature dropped
    '''
    dropped_df = df.drop(columns=cat_feature)
    return dropped_df

In [11]:
# normalize the features
def normalize(df):
    '''
    Params:
        df -> pandas dataframe: the original dataframe for recommendation
    Returns:
        normalized_df -> pandas dataframe: the dataframe with normalized 
                                           features for recommendation
    '''
    try:
        columns = list(df.columns)
        normalized_df = df
        for column in columns:
            if column != 'artificial_listing_id' and column != 'rexUrl':
                col = df[column]
                normalized_df[column] = (col-col.min())/(col.max()-col.min())
        return normalized_df
    except ZeroDivisionError:
        return df

In [12]:
df_for_recom = merge_df2[selected_features]
df_for_recom.head()

,ip,country_code2,eventAttributes,latitude_y,longitude_y,city,state,full_baths,bedrooms,land_square_footage,...,marketing_budget,property_type,school_district_name,status,stories,total_baths,year_built,call_ct,buyer_call,rexUrl
0,74.125.150.82,US,"{'city': 'Jacksonville', 'preferred_baths_rang...",30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,MEDIUM,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville
1,74.125.150.82,US,NaN,30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,MEDIUM,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville
2,74.125.150.82,US,NaN,30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,MEDIUM,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville
3,74.125.150.82,US,"{'city': 'Laurel', 'preferred_baths_range': '[...",39.131037,-76.837090,Laurel,MD,3.0,3.0,0.0,...,MEDIUM,TOWNHOUSE,Howard County Public Schools,SOLD,2.0,3.5,1989.0,NaN,NaN,9613-glendower-ct
4,74.125.150.82,US,NaN,39.131037,-76.837090,Laurel,MD,3.0,3.0,0.0,...,MEDIUM,TOWNHOUSE,Howard County Public Schools,SOLD,2.0,3.5,1989.0,NaN,NaN,9613-glendower-ct


In [13]:
listings_data = df_for_recom.drop(columns=['ip','eventAttributes']).drop_duplicates()
listings_data.head()

,country_code2,latitude_y,longitude_y,city,state,full_baths,bedrooms,land_square_footage,living_square_feet,marketing_budget,property_type,school_district_name,status,stories,total_baths,year_built,call_ct,buyer_call,rexUrl
0,US,30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,1108.0,MEDIUM,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville
3,US,39.131037,-76.837090,Laurel,MD,3.0,3.0,0.0,1560.0,MEDIUM,TOWNHOUSE,Howard County Public Schools,SOLD,2.0,3.5,1989.0,NaN,NaN,9613-glendower-ct
6,US,28.954697,-81.314121,Orange City,FL,2.0,3.0,10350.0,1677.0,MEDIUM,SINGLE,Volusia County School District,FOR_SALE,1.0,2.0,2004.0,NaN,NaN,594-perry-st
15,US,29.452820,-98.369925,San Antonio,TX,2.0,3.0,5018.0,2191.0,MEDIUM,SINGLE,Judson Independent School District,FOR_SALE,2.0,2.5,2017.0,NaN,NaN,2527-live-oak-pass
18,US,30.690586,-97.730705,Georgetown,TX,3.0,4.0,7405.0,2423.0,MEDIUM,SINGLE,Georgetown Independent School District,FOR_SALE,1.0,2.0,2017.0,NaN,NaN,1224-lakeside-ranch-rd


In [14]:
# join the one-hot encoded features dataframe with original one
for feature in cat_features:
    transformed_df = transformer(listings_data, feature)
    listings_data = join(listings_data, transformed_df)

# drop the original categorical features
for feature in cat_features:
    listings_data = drop(listings_data, feature)


In [15]:
listings_data.head(5)

,latitude_y,longitude_y,full_baths,bedrooms,land_square_footage,living_square_feet,stories,total_baths,year_built,call_ct,...,school_district_name_Yucaipa-Calimesa Joint Unified School District,status_ACCEPTING_BACKUP_OFFERS,status_CLAIMED,status_COMING_SOON,status_FOR_SALE,status_INACTIVE,status_IN_ESCROW,status_PREVIEW,status_SIGNED,status_SOLD
0,30.345093,-81.550354,2.0,2.0,1360.0,1108.0,2.0,2.5,2006.0,3.0,...,0,0,0,0,0,0,0,0,0,1
3,39.131037,-76.837090,3.0,3.0,0.0,1560.0,2.0,3.5,1989.0,NaN,...,0,0,0,0,0,0,0,0,0,1
6,28.954697,-81.314121,2.0,3.0,10350.0,1677.0,1.0,2.0,2004.0,NaN,...,0,0,0,0,1,0,0,0,0,0
15,29.452820,-98.369925,2.0,3.0,5018.0,2191.0,2.0,2.5,2017.0,NaN,...,0,0,0,0,1,0,0,0,0,0
18,30.690586,-97.730705,3.0,4.0,7405.0,2423.0,1.0,2.0,2017.0,NaN,...,0,0,0,0,1,0,0,0,0,0


In [16]:
# use longitude and latitude as indicator for unique housing listing
# here we create an artificial unique housing listing id
artificial_listing_id = range(len(listings_data))
listings_data['artificial_listing_id'] = artificial_listing_id

df_for_recom = df_for_recom.merge(listings_data[['latitude_y','longitude_y','artificial_listing_id']],\
                                  on=['latitude_y','longitude_y'], how='left')
df_for_recom.head()

,ip,country_code2,eventAttributes,latitude_y,longitude_y,city,state,full_baths,bedrooms,land_square_footage,...,property_type,school_district_name,status,stories,total_baths,year_built,call_ct,buyer_call,rexUrl,artificial_listing_id
0,74.125.150.82,US,"{'city': 'Jacksonville', 'preferred_baths_rang...",30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville,0
1,74.125.150.82,US,NaN,30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville,0
2,74.125.150.82,US,NaN,30.345093,-81.550354,Jacksonville,FL,2.0,2.0,1360.0,...,TOWNHOUSE,Duval,SOLD,2.0,2.5,2006.0,3.0,0.0,1511-fieldview-dr-jacksonville,0
3,74.125.150.82,US,"{'city': 'Laurel', 'preferred_baths_range': '[...",39.131037,-76.837090,Laurel,MD,3.0,3.0,0.0,...,TOWNHOUSE,Howard County Public Schools,SOLD,2.0,3.5,1989.0,NaN,NaN,9613-glendower-ct,1
4,74.125.150.82,US,NaN,39.131037,-76.837090,Laurel,MD,3.0,3.0,0.0,...,TOWNHOUSE,Howard County Public Schools,SOLD,2.0,3.5,1989.0,NaN,NaN,9613-glendower-ct,1


Content based model:

first calculate the cosine similarity matrix with listings_data
for a user (suppose ip address indicates a unique user),
given the listings l_u the user has clicked,
select the other listings l_o within the range according to eventAttributes;
for each such listing,
calculate the average cosine similarity of l_o over l_u,
then recommend the listings with the top cosine similarities.

note: eventAttributes has restriction on
city, preferred_baths_range (e.g. [2.5, 2.5]), country, region (e.g. FL), 
preferred_beds_range (e.g. [2, 2]), currency (e.g. USD), property_type.


In [18]:
# indices = pd.Series(listings_data.index, index=listings_data['artificial_listing_id'])
listings_data = listings_data.drop_duplicates(subset='artificial_listing_id', keep="first")
listings_data.reset_index(drop=True, inplace=True)
listings_data = normalize(listings_data)
indices = pd.Series(listings_data.index, index=listings_data['artificial_listing_id'])


In [20]:
listings_data_no_rexurl = listings_data.drop(columns=['rexUrl'])

In [21]:
def calculate_cosine_similarity(listingID, df=listings_data):
    '''
    A method that returns the most similar listings to the current list

    Params:
        listingID -> int: artificial_listing_id of the dataset
        df -> pandas dataframe: the dataset containing the listing data

    Return:
        The listed homes with cosine similarity scores
    '''
    # index record
    idx = indices[listingID]
    # print(indices.shape)
    # print(idx)
    # print('again')

    # fill null values with 0
    df.fillna(0, inplace=True)

    # do not include artificial listing id as a feature
    new = df.drop(columns=['artificial_listing_id'])
    # calculate the cosine similarity matrix
    similar = cosine_similarity(new.values)

    # put the correlation back into the original dataset
    df['correlation'] = similar[idx]
    # corr_similar = pd.DataFrame(df)

    # sort the rows by the correlation score
    result = df.sort_values(by='correlation', ascending=False)

    return result


In [22]:
calculate_cosine_similarity(3, listings_data_no_rexurl)

,latitude_y,longitude_y,full_baths,bedrooms,land_square_footage,living_square_feet,stories,total_baths,year_built,call_ct,...,status_CLAIMED,status_COMING_SOON,status_FOR_SALE,status_INACTIVE,status_IN_ESCROW,status_PREVIEW,status_SIGNED,status_SOLD,artificial_listing_id,correlation
3,0.166085,0.480880,0.125,0.190476,0.002880,0.014712,0.111111,0.060976,0.988889,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3,1.000000
100,0.176603,0.480344,0.250,0.190476,0.003704,0.019591,0.055556,0.067073,0.981481,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,100,0.998863
2250,0.166085,0.480880,0.125,0.190476,0.002880,0.014712,0.111111,0.060976,0.988889,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2250,0.879590
1896,0.176603,0.480344,0.250,0.190476,0.003704,0.019591,0.055556,0.067073,0.981481,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1896,0.878648
634,0.177712,0.478738,0.375,0.333333,0.037465,0.035393,0.111111,0.121951,0.951852,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,634,0.875295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,0.751193,0.936263,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2309,0.188112
2238,0.747868,0.935759,0.000,0.142857,0.000000,0.006278,0.000000,0.024390,0.777778,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2238,0.167700
1870,0.336396,0.130830,0.375,0.238095,0.004017,0.018354,0.111111,0.097561,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1870,0.157381
1116,0.421790,0.106009,0.000,0.142857,0.029745,0.004028,0.111111,0.024390,0.888889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1116,0.136426


In [23]:
calculate_cosine_similarity(0, listings_data_no_rexurl)[['correlation', 'artificial_listing_id']]

,correlation,artificial_listing_id
0,1.000000,0
173,0.878822,173
751,0.773613,751
344,0.773333,344
75,0.773283,75
...,...,...
802,0.161381,802
2054,0.160820,2054
1901,0.160820,1901
2035,0.160788,2035


In [24]:
# store the cosine similarities into a dictionary
cosine_similarity_matrix_dict = {}
for i in range(len(listings_data)):
    similarity_dict = calculate_cosine_similarity(i, listings_data_no_rexurl)[['correlation', 'artificial_listing_id']].set_index('artificial_listing_id')['correlation'].to_dict()
    cosine_similarity_matrix_dict[i] = similarity_dict

In [25]:
def apply_event_attributes(ip, df):
    '''
    A method that selects the listings matching with the eventAttributes for a specific user

    Params:
        ip -> str: the ip address that indicates a specific user
        df -> pandas dataframe: the original dataset that contains the user activities
    
    Returns:
        The selected listings (artificial listing ids) that match with 
        the eventAttributes for a specific user
    '''
    user_activity = df[df['ip'] == ip]
    event_attributes = list(user_activity['eventAttributes'])

    # initialize the preferences
    preferred_cities = []
    preferred_bath_range = [10000, 0]
    preferred_regions = []
    preferred_bed_range = [10000, 0]
    preferred_property_types = []

    # finalize the event attributes
    for attributes in event_attributes:
        if attributes is not np.nan:
            attributes = ast.literal_eval(attributes)

            preferred_cities.append(attributes.get('city'))
            preferred_regions.append(attributes.get('region'))
            preferred_property_types.append(attributes.get('property_type'))

            try:
                preferred_bath_range_attr = attributes.get('preferred_baths_range')
                if preferred_bath_range_attr is not None:
                    lower_bath_num = json.loads(preferred_bath_range_attr)[0]
                    upper_bath_num = json.loads(preferred_bath_range_attr)[1]
                    if lower_bath_num is not None:
                        preferred_bath_range[0] = min(preferred_bath_range[0], float(lower_bath_num))
                    if upper_bath_num is not None:
                        preferred_bath_range[1] = max(preferred_bath_range[1], float(upper_bath_num))
            except:
                pass
            
            try:
                preferred_bed_range_attr = attributes.get('preferred_beds_range')
                if preferred_bed_range_attr is not None:
                    lower_bed_num = json.loads(preferred_bed_range_attr)[0]
                    upper_bed_num = json.loads(preferred_bed_range_attr)[1]
                    if lower_bed_num is not None:
                        preferred_bed_range[0] = min(preferred_bed_range[0], float(lower_bed_num))
                    if upper_bed_num is not None:
                        preferred_bed_range[1] = max(preferred_bed_range[1], float(upper_bed_num))
            except:
                pass
    
    preferred_cities = set(preferred_cities)
    preferred_regions = set(preferred_regions)
    preferred_property_types = set(preferred_property_types)

    # select the dataframe with filtered attributes
    filtered_df = df[df['city'].isin(preferred_cities)][df['state'].isin(preferred_regions)][df['property_type'].isin(preferred_property_types)][df['total_baths']>preferred_bath_range[0]][df['total_baths']<preferred_bath_range[1]][df['bedrooms']>preferred_bed_range[0]][df['bedrooms']<preferred_bed_range[1]]

    # return the artificial listing ids
    result = set(filtered_df['artificial_listing_id'])
    # if there is no filtering or there is no satisfactory results
    if len(result) == 0:
        # number of listings is 2348
        return list(range(2348))
    return list(result)


In [27]:
def recommender(ip, df):
    '''
    A method that caclulates an average cosine similarity score for each listing
    in the search list (determined by event_attributes) and recommends the top listing

    Params:

    Return:
        result -> The top listing that's considered the most similar to the users' previous history
    '''
    search_result = apply_event_attributes(ip, df)
    user_activity = df[df['ip'] == ip]
    user_activity =df.drop_duplicates(subset='artificial_listing_id', keep="first")
    listings= list(user_activity['artificial_listing_id'])

    result = []
    for i in search_result:
        scores  = []
        for listing in listings:
            scores.append(cosine_similarity_matrix_dict[i][listing])
        average_score = np.mean(np.array(scores))
        result.append([i, average_score])

    result = pd.DataFrame(result, columns=["artificial id", "correlation"])
    result = result.sort_values(by='correlation', ascending=False)
    return result


In [28]:
def evaluation(ip, df):
    '''
    A method that caclulates an average cosine similarity score for each listing
    in the search list (determined by event_attributes) and recommends the top listing

    Params:
        ip -> str: the ip address that indicates a specific user
        df -> pandas dataframe: the original dataset that contains the user activities
    Return:
        result -> The top listing that's considered the most similar to the users' previous history (n-1 listings)
        actual -> The actual listing that that the user has viewed (the n-th listing)
    '''
    search_result = apply_event_attributes(ip, df)
    user_activity = df[df['ip'] == ip]
    user_activity =df.drop_duplicates(subset='artificial_listing_id', keep="first")
    listings= list(user_activity['artificial_listing_id'])

    result = []
    for i in search_result:
        scores  = []
        # use n-1 listings to predict the n-th listing
        for listing in listings[:-1]:
            scores.append(cosine_similarity_matrix_dict[i][listing])
        average_score = np.mean(np.array(scores))
        result.append([i, average_score])

    result = pd.DataFrame(result, columns=["artificial id", "correlation"])
    result = result.sort_values(by='correlation', ascending=False)
    return result.iloc[0]["artificial id"], listings[-1]


In [29]:
recommender(ip='74.125.150.86', df=df_for_recom)

,artificial id,correlation
781,1237,0.588212
788,1253,0.587026
957,1591,0.586389
419,507,0.586345
884,1449,0.585786
...,...,...
1280,2255,0.285402
1082,1870,0.250231
809,1293,0.217143
1079,1863,0.217054


In [45]:
listings_data[listings_data['artificial_listing_id']==1237]

,latitude_y,longitude_y,full_baths,bedrooms,land_square_footage,living_square_feet,stories,total_baths,year_built,call_ct,...,status_ACCEPTING_BACKUP_OFFERS,status_CLAIMED,status_COMING_SOON,status_FOR_SALE,status_INACTIVE,status_IN_ESCROW,status_PREVIEW,status_SIGNED,status_SOLD,artificial_listing_id
1237,0.738484,0.056066,0.222222,0.285714,0.00425,0.021638,0.111111,0.073171,0.991584,0.104167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1237


In [46]:
listings_data[listings_data['artificial_listing_id']==1237].rexUrl

1237    30953-young-dove-st
Name: rexUrl, dtype: object

In [40]:
df_for_recom[df_for_recom['ip']=='74.125.150.86'][df_for_recom['state']=='CA']

,ip,country_code2,eventAttributes,latitude_y,longitude_y,city,state,full_baths,bedrooms,land_square_footage,...,property_type,school_district_name,status,stories,total_baths,year_built,call_ct,buyer_call,rexUrl,artificial_listing_id
57,74.125.150.86,US,"{'city': 'Fair Oaks', 'preferred_baths_range':...",38.647052,-121.279172,Fair Oaks,CA,2.0,3.0,20038.0,...,SINGLE,San Juan Unified School District,SOLD,2.0,2.0,1974.0,NaN,NaN,7711-oak-leaf-ave,7
58,74.125.150.86,US,NaN,38.647052,-121.279172,Fair Oaks,CA,2.0,3.0,20038.0,...,SINGLE,San Juan Unified School District,SOLD,2.0,2.0,1974.0,NaN,NaN,7711-oak-leaf-ave,7
59,74.125.150.86,US,NaN,38.647052,-121.279172,Fair Oaks,CA,2.0,3.0,20038.0,...,SINGLE,San Juan Unified School District,SOLD,2.0,2.0,1974.0,NaN,NaN,7711-oak-leaf-ave,7
177,74.125.150.86,US,"{'city': 'Rancho Cordova', 'preferred_baths_ra...",38.608485,-121.278656,Rancho Cordova,CA,2.0,3.0,6970.0,...,SINGLE,Folsom-Cordova Unified School District,IN_ESCROW,1.0,2.0,1977.0,NaN,NaN,2420-berrywood-dr,15
178,74.125.150.86,US,"{'city': 'Rancho Cordova', 'preferred_baths_ra...",38.608485,-121.278656,Rancho Cordova,CA,2.0,3.0,6970.0,...,SINGLE,Folsom-Cordova Unified School District,IN_ESCROW,1.0,2.0,1977.0,NaN,NaN,2420-berrywood-dr,2277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276121,74.125.150.86,US,NaN,34.022143,-118.471205,Santa Monica,CA,3.0,3.0,10772.0,...,CONDO,NaN,SOLD,2.0,2.5,1981.0,NaN,NaN,2022-delaware-ave-apt-4,572
1276122,74.125.150.86,US,NaN,34.022143,-118.471205,Santa Monica,CA,3.0,3.0,10772.0,...,CONDO,NaN,SOLD,2.0,2.5,1981.0,NaN,NaN,2022-delaware-ave-apt-4,572
1276129,74.125.150.86,US,"{'city': 'Oak Park', 'preferred_baths_range': ...",34.183730,-118.765743,Oak Park,CA,2.0,2.0,618.0,...,CONDO,Oak Park Unified School District,SOLD,2.0,2.5,1990.0,2.0,0.0,5796-freebird-ln-unit-108,1412
1276130,74.125.150.86,US,NaN,34.183730,-118.765743,Oak Park,CA,2.0,2.0,618.0,...,CONDO,Oak Park Unified School District,SOLD,2.0,2.5,1990.0,2.0,0.0,5796-freebird-ln-unit-108,1412


In [35]:
recommender(ip='74.125.150.82', df=df_for_recom)

,artificial id,correlation
392,507,0.622786
220,254,0.622496
819,1449,0.622237
764,1335,0.622132
884,1585,0.622071
...,...,...
996,1870,0.286724
749,1293,0.253570
1008,1886,0.251159
993,1863,0.250523


In [30]:
# randomly sample ip addresses for evaluation
all_ips = list(df_for_recom.ip.unique())
ips_for_eval = sample(all_ips, 2000)

In [48]:
# evaluate the recommender
evaluate_result = {}
for ip in ips_for_eval:
    predict_listing_id, actual_listing_id = evaluation(ip, df_for_recom)
    predict_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==predict_listing_id]
    actual_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==actual_listing_id]
    similarity_score = cosine_similarity(predict_listing.values, actual_listing.values)
    evaluate_result[ip] = similarity_score
evaluate_result

{'172.56.26.100': array([[0.99999858]]),
 '172.58.22.144': array([[0.99999858]]),
 '73.72.197.235': array([[0.99999858]]),
 '99.203.69.130': array([[0.99999858]]),
 '72.89.35.157': array([[0.99999858]]),
 '97.96.58.228': array([[0.99999858]]),
 '68.82.33.130': array([[0.99999858]]),
 '24.0.97.181': array([[0.99999858]]),
 '66.138.182.248': array([[0.99999858]]),
 '174.238.139.24': array([[0.99999858]]),
 '174.57.55.42': array([[0.99999858]]),
 '47.156.169.39': array([[0.99999858]]),
 '99.203.55.126': array([[0.99999858]]),
 '24.99.207.93': array([[0.99999858]]),
 '172.58.235.170': array([[0.99999858]]),
 '198.241.174.12': array([[0.99999858]]),
 '73.81.150.205': array([[0.99999858]]),
 '172.14.125.222': array([[0.99999858]]),
 '99.203.80.148': array([[0.99999858]]),
 '174.193.146.175': array([[0.99999858]]),
 '47.144.134.185': array([[0.99999858]]),
 '172.58.227.161': array([[0.99955176]]),
 '24.188.15.39': array([[0.99999858]]),
 '162.205.52.32': array([[0.99999858]]),
 '100.12.200.13

In [49]:
evaluate_average_similarity_score = np.mean(np.array([score[0][0] for score in list(evaluate_result.values())]))
print("The average cosine similarity score of the evaluation data is", evaluate_average_similarity_score)

The average cosine similarity score of the evaluation data is 0.9999950596834458


In [ ]:
# evaluate the recommender
evaluate_result = {}
for ip in ips_for_eval:
    predict_listing_id, actual_listing_id = evaluation(ip, df_for_recom)
    predict_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==predict_listing_id]
    actual_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==actual_listing_id]
    similarity_score = cosine_similarity(predict_listing.values, actual_listing.values)
    evaluate_result[ip] = similarity_score
evaluate_result

In [50]:
overall_score = 0
i = 0
for listing_id, score_dict in cosine_similarity_matrix_dict.items():
    for key, value in score_dict.items():
        overall_score += value
        i += 1
print("The average similarity score of all data is", overall_score/i)

The average similarity score of all data is 0.49552892667940174


In [39]:
scores = []
for ip in ips_for_eval:
    search_result = apply_event_attributes(ip, df_for_recom)
    user_activity = df_for_recom[df_for_recom['ip'] == ip]
    user_activity =df_for_recom.drop_duplicates(subset='artificial_listing_id', keep="first")
    listings= list(user_activity['artificial_listing_id'])
    
    cosine_scores = [cosine_similarity_matrix_dict[i][listings[-1]] for i in listings[:-1]]
    scores.append(sum(cosine_scores)/len(cosine_scores))
print("ground truth:", sum(scores)/len(scores))

ground truth: 0.5251023996177304


In [44]:
# evaluate the recommender with complete dataset
complete_evaluate_result = {}
for ip in all_ips:
    predict_listing_id, actual_listing_id = evaluation(ip, df_for_recom)
    predict_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==predict_listing_id]
    actual_listing = listings_data_no_rexurl[listings_data_no_rexurl['artificial_listing_id']==actual_listing_id]
    similarity_score = cosine_similarity(predict_listing.values, actual_listing.values)
    complete_evaluate_result[ip] = similarity_score
complete_evaluate_result

{'74.125.150.82': array([[0.95012992]]),
 '74.125.150.86': array([[0.95012992]]),
 '74.125.150.90': array([[0.94053715]]),
 '74.125.150.88': array([[0.95012992]]),
 '74.125.150.84': array([[0.95012992]]),
 '74.125.150.81': array([[0.95012992]]),
 '74.125.150.83': array([[0.95012992]]),
 '74.125.150.85': array([[0.95012992]]),
 '72.14.199.241': array([[0.95012992]]),
 '72.14.199.245': array([[0.95012992]]),
 '72.14.199.243': array([[0.95012992]]),
 '72.14.199.240': array([[0.95012992]]),
 '72.14.199.244': array([[0.94594174]]),
 '72.14.199.242': array([[0.93729749]]),
 '72.14.199.247': array([[0.94053715]]),
 '72.14.199.249': array([[0.95848943]]),
 '73.131.176.139': array([[0.95012992]]),
 '71.105.99.225': array([[0.95012992]]),
 '72.226.6.187': array([[0.77597283]]),
 '172.9.163.61': array([[0.97530856]]),
 '24.188.136.169': array([[0.95012992]]),
 '68.192.229.143': array([[0.95012992]]),
 '71.167.20.218': array([[0.95012992]]),
 '68.173.182.146': array([[0.95782602]]),
 '173.2.193.46

In [45]:
complete_evaluate_score = np.mean(np.array([score[0][0] for score in list(complete_evaluate_result.values())]))
print("The average cosine similarity score of the evaluation data (complete dataset) is", complete_evaluate_score)


The average cosine similarity score of the evaluation data (complete dataset) is 0.946281462010624
